In [1]:
import geopandas as gpd
from geopandas.tools import overlay
import os
import matplotlib.pyplot as plt
import boto3
import zipfile
import sys
import xarray as xr
import pandas as pd
from zipfile import ZipFile



In [2]:
import os
import shutil
import boto3
from zipfile import ZipFile
import geopandas as gpd

# Initialize the S3 client
s3_client = boto3.client('s3')

# Bucket name and file paths
bucket_name = 'ca-climate-index'
directory = '1_pull_data/built_environment/communication_infrastructure/homeland_infrastructure_foundation_level_data/TV_Broadcast_Contours.zip'
out_directory = '2a_subset/built_environment/communication_infrastructure/homeland_infrastructure_foundation_level_data/'

# Local directory to store the downloaded zip file and extracted contents
local_directory = 'temp'
if not os.path.exists(local_directory):
    os.makedirs(local_directory)

# Download the zip file
local_zip_file_path = os.path.join(local_directory, os.path.basename(directory))
s3_client.download_file(bucket_name, directory, local_zip_file_path)

# Extract the contents of the zip file
with ZipFile(local_zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(local_directory)

# Manipulate the shapefile within the extracted contents
# Assuming there's only one shapefile in the zip file
shapefile_path = None
for filename in os.listdir(local_directory):
    if filename.endswith('.shp'):
        shapefile_path = os.path.join(local_directory, filename)
        # Perform your manipulation with the shapefile using geopandas or other libraries
        # For example, you can read it using geopandas
        gdf = gpd.read_file(shapefile_path)
        sta_gdf = gdf[gdf['STATUS'] == 'STA']
        
        # Create a directory to store the shapefile and its associated files
        output_folder = os.path.join(local_directory, 'output_shapefile')
        os.makedirs(output_folder, exist_ok=True)
        
        # Save the filtered shapefile inside the output folder
        output_shapefile_path = os.path.join(output_folder, 'sta_tv_contours.shp')
        sta_gdf.to_file(output_shapefile_path)
        
        # Zip the output folder
        output_zip_file_path = os.path.join(local_directory, 'sta_tv_contours.zip')
        shutil.make_archive(output_zip_file_path[:-4], 'zip', output_folder)
        
        # Upload the zipped folder to AWS S3
        s3_client.upload_file(output_zip_file_path, bucket_name, os.path.join(out_directory, 'sta_tv_contours.zip'))

# Clean up temporary files
os.remove(local_zip_file_path)  # Remove the downloaded zip file
shutil.rmtree(local_directory)  # Remove the extracted files except the new shapefile


In [5]:
display(sta_gdf.STATUS)

13       STA
14       STA
15       STA
218      STA
298      STA
        ... 
33328    STA
33373    STA
33405    STA
33451    STA
33531    STA
Name: STATUS, Length: 2282, dtype: object